## 1. Biblioteki

In [2]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import os
import talos as ta
from talos.model.early_stopper import early_stopper

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error
%load_ext tensorboard


from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns

from sklearn.model_selection import train_test_split


Using TensorFlow backend.



## 2. Dane

In [3]:
train_df=pd.read_csv('D:/STUDIA/ROK_II/Projekt/Dane_jalowka_i_krowa_train.csv')
test_df=pd.read_csv('D:/STUDIA/ROK_II/Projekt/Dane_jalowka_i_krowa_test.csv')
val_df=pd.read_csv('D:/STUDIA/ROK_II/Projekt/Dane_jalowka_i_krowa_val.csv')

train_df.columns = ['id','Województwo', 'Data urodzenia', 'IE', 'Dokł. IE', 'PF', 'PI-PROD', 'PI-POKR', 'Prc', 'Psm', 'Pnr', 'Pw','PI-PLOD', 'CRj', 'CRk', 'PP', 'OMC', 'WH-KSOM', 'WH-DLUG', 'IP', 'kg ml', 'kg tł', '% tł', 'kg bi', '% bi', 'rc', 'sm', 'nr', 'w', 'og','kategoria']
test_df.columns = ['id','Województwo', 'Data urodzenia', 'IE', 'Dokł. IE', 'PF', 'PI-PROD', 'PI-POKR', 'Prc', 'Psm', 'Pnr', 'Pw','PI-PLOD', 'CRj', 'CRk', 'PP', 'OMC', 'WH-KSOM', 'WH-DLUG', 'IP', 'kg ml', 'kg tł', '% tł', 'kg bi', '% bi', 'rc', 'sm', 'nr', 'w', 'og','kategoria']
val_df.columns = ['id','Województwo', 'Data urodzenia', 'IE', 'Dokł. IE', 'PF', 'PI-PROD', 'PI-POKR', 'Prc', 'Psm', 'Pnr', 'Pw','PI-PLOD', 'CRj', 'CRk', 'PP', 'OMC', 'WH-KSOM', 'WH-DLUG', 'IP', 'kg ml', 'kg tł', '% tł', 'kg bi', '% bi', 'rc', 'sm', 'nr', 'w', 'og','kategoria']

caly_df = pd.concat([train_df, test_df, val_df])
caly_df['Województwo']=caly_df['Województwo'].astype('category')
caly_df['Data urodzenia']=caly_df['Data urodzenia'].astype('category')

del caly_df["id"]
del caly_df["PF"]
del caly_df["Dokł. IE"]
del caly_df['kategoria']

caly_df.shape

(1881, 27)

In [4]:
caly_df.head()

,Województwo,Data urodzenia,IE,PI-PROD,PI-POKR,Prc,Psm,Pnr,Pw,PI-PLOD,...,kg ml,kg tł,% tł,kg bi,% bi,rc,sm,nr,w,og
0,WIELKOPOLSKIE,2020,2120,137,127,121,113,106,128,109,...,1370,577,2,452,1,115.0,114.0,109.0,127.0,123.0
1,KUJAWSKO-POMORSKIE,2018,1912,126,123,109,113,103,128,114,...,1209,358,-16,354,-4,115.0,113.0,104.0,123.0,123.0
2,POMORSKIE,2020,2126,126,120,110,107,102,124,113,...,982,368,-4,350,4,104.0,103.0,102.0,122.0,117.0
3,KUJAWSKO-POMORSKIE,2020,2089,133,111,104,110,99,114,114,...,1080,546,12,410,7,106.0,106.0,101.0,112.0,112.0
4,ZACHODNIOPOMORSKIE,2020,2853,125,115,91,95,101,124,134,...,516,501,36,289,15,102.0,102.0,118.0,123.0,109.0


In [5]:
caly_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1881 entries, 0 to 188
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Województwo     1881 non-null   category
 1   Data urodzenia  1881 non-null   category
 2   IE              1881 non-null   int64   
 3   PI-PROD         1881 non-null   int64   
 4   PI-POKR         1881 non-null   int64   
 5   Prc             1881 non-null   int64   
 6   Psm             1881 non-null   int64   
 7   Pnr             1881 non-null   int64   
 8   Pw              1881 non-null   int64   
 9   PI-PLOD         1881 non-null   int64   
 10  CRj             1881 non-null   int64   
 11  CRk             1881 non-null   int64   
 12  PP              1881 non-null   int64   
 13  OMC             1881 non-null   int64   
 14  WH-KSOM         1881 non-null   int64   
 15  WH-DLUG         1881 non-null   int64   
 16  IP              1881 non-null   int64   
 17  kg ml          

## 3. Usuwam mniejszości zmiennych kategorycznych

In [6]:
caly_df['Województwo'].value_counts()

WIELKOPOLSKIE          579
KUJAWSKO-POMORSKIE     298
OPOLSKIE               227
ZACHODNIOPOMORSKIE     212
PODLASKIE              175
MAZOWIECKIE            106
POMORSKIE               94
LUBUSKIE                47
ŁÓDZKIE                 41
DOLNOŚLĄSKIE            32
WARMIŃSKO-MAZURSKIE     23
ŚLĄSKIE                 17
ŚWIĘTOKRZYSKIE          13
MAŁOPOLSKIE             12
LUBELSKIE                4
PODKARPACKIE             1
Name: Województwo, dtype: int64

In [7]:
caly_df=caly_df[caly_df['Województwo']!='PODKARPACKIE']
caly_df=caly_df[caly_df['Województwo']!='LUBELSKIE']

In [8]:
caly_df['Województwo'].value_counts()

WIELKOPOLSKIE          579
KUJAWSKO-POMORSKIE     298
OPOLSKIE               227
ZACHODNIOPOMORSKIE     212
PODLASKIE              175
MAZOWIECKIE            106
POMORSKIE               94
LUBUSKIE                47
ŁÓDZKIE                 41
DOLNOŚLĄSKIE            32
WARMIŃSKO-MAZURSKIE     23
ŚLĄSKIE                 17
ŚWIĘTOKRZYSKIE          13
MAŁOPOLSKIE             12
LUBELSKIE                0
PODKARPACKIE             0
Name: Województwo, dtype: int64

In [9]:
caly_df["IE"].describe()

count    1876.000000
mean     2186.273454
std       166.944364
min      1903.000000
25%      2096.000000
50%      2155.500000
75%      2243.000000
max      3165.000000
Name: IE, dtype: float64

In [10]:
caly_df["OMC"].describe()

count    1876.000000
mean      107.656183
std         7.421239
min        82.000000
25%       103.000000
50%       107.000000
75%       112.000000
max       132.000000
Name: OMC, dtype: float64

In [9]:
caly_df[caly_df['Województwo']=='WIELKOPOLSKIE']['IE'].describe()

count     579.000000
mean     2191.430052
std       164.271833
min      1905.000000
25%      2098.000000
50%      2158.000000
75%      2248.000000
max      2814.000000
Name: IE, dtype: float64

In [10]:
caly_df[caly_df['Województwo']=='KUJAWSKO-POMORSKIE']['IE'].describe()

count     298.000000
mean     2204.234899
std       185.627080
min      1908.000000
25%      2094.000000
50%      2157.000000
75%      2279.000000
max      2981.000000
Name: IE, dtype: float64

In [11]:
caly_df[caly_df['Województwo']=='POMORSKIE']['IE'].describe()

count      94.000000
mean     2148.553191
std       130.931946
min      1903.000000
25%      2093.000000
50%      2126.000000
75%      2213.000000
max      2566.000000
Name: IE, dtype: float64

In [12]:
caly_df[caly_df['Województwo']=='PODLASKIE']['IE'].describe()

count     175.000000
mean     2172.045714
std       177.272529
min      1903.000000
25%      2094.000000
50%      2139.000000
75%      2217.000000
max      3165.000000
Name: IE, dtype: float64

In [24]:
caly_df[caly_df['Województwo']=='ŁÓDZKIE']['OMC'].describe()

count     41.000000
mean     108.121951
std        8.594170
min       96.000000
25%      101.000000
50%      108.000000
75%      114.000000
max      128.000000
Name: OMC, dtype: float64

In [14]:
caly_df[caly_df['Województwo']=='OPOLSKIE']['IE'].describe()

count     227.000000
mean     2175.352423
std       164.975431
min      1909.000000
25%      2089.500000
50%      2160.000000
75%      2230.000000
max      2939.000000
Name: IE, dtype: float64

In [15]:
caly_df[caly_df['Województwo']=='LUBUSKIE']['IE'].describe()

count      47.000000
mean     2188.659574
std       140.108231
min      1941.000000
25%      2108.500000
50%      2152.000000
75%      2252.000000
max      2580.000000
Name: IE, dtype: float64

In [16]:
caly_df[caly_df['Województwo']=='DOLNOŚLĄSKIE']['IE'].describe()

count      32.000000
mean     2193.281250
std       181.702208
min      1903.000000
25%      2085.000000
50%      2164.000000
75%      2291.000000
max      2706.000000
Name: IE, dtype: float64

In [17]:
caly_df[caly_df['Województwo']=='MAZOWIECKIE']['IE'].describe()

count     106.000000
mean     2167.801887
std       161.987385
min      1905.000000
25%      2087.500000
50%      2137.000000
75%      2216.500000
max      2718.000000
Name: IE, dtype: float64

In [18]:
caly_df[caly_df['Województwo']=='WARMIŃSKO-MAZURSKIE']['IE'].describe()

count      23.000000
mean     2121.956522
std       105.500745
min      1904.000000
25%      2085.000000
50%      2125.000000
75%      2178.000000
max      2341.000000
Name: IE, dtype: float64

In [22]:
caly_df[caly_df['Województwo']=='ŚLĄSKIE']['OMC'].describe()

count     17.000000
mean     109.647059
std        7.607079
min      100.000000
25%      103.000000
50%      109.000000
75%      116.000000
max      125.000000
Name: OMC, dtype: float64

In [23]:
caly_df[caly_df['Województwo']=='ŚWIĘTOKRZYSKIE']['OMC'].describe()

count     13.000000
mean     112.230769
std        6.365975
min      102.000000
25%      107.000000
50%      112.000000
75%      118.000000
max      121.000000
Name: OMC, dtype: float64

In [21]:
caly_df[caly_df['Województwo']=='MAŁOPOLSKIE']['IE'].describe()

count      12.000000
mean     2165.583333
std       136.253413
min      1903.000000
25%      2122.500000
50%      2177.000000
75%      2259.250000
max      2345.000000
Name: IE, dtype: float64

In [1]:
caly_df['IE'].describe()

In [ ]:
caly_df['OMC'].describe()